## SageMaker Jumpstart

- tabular, time series, image, text 등 다양한 task에 대한 모델을 가지고 있습니다.
- 기본적인 부분은 sagemaker의 built-in algorithm과 비슷합니다.
- 최근에 많이 추가된 기능은 Foundation model 과 관련된 내용들로, Large Language Model, Generative AI 등을 쉽게 사용할 수가 있습니다.


### Stable diffusion fine tuning 진행

- 내가 원하는 이미지를 생성하는 예시
- 블로그 참고 : https://aws.amazon.com/ko/blogs/machine-learning/fine-tune-text-to-image-stable-diffusion-models-with-amazon-sagemaker-jumpstart/
- 예시 노트북 : https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_amazon_algorithms/jumpstart_text_to_image/Amazon_JumpStart_Text_To_Image.ipynb
  - 데이터 준비 예시 : https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_amazon_algorithms/jumpstart_text_to_image/custom_dog_image_generator.ipynb



In [ ]:
import sagemaker, boto3, json
from sagemaker.session import Session
from sagemaker import image_uris, model_uris, script_uris
from sagemaker.estimator import Estimator
from sagemaker import hyperparameters

sagemaker_session = Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
bucket = sagemaker_session.default_bucket()
print(bucket)

In [ ]:
# Currently, not all the stable diffusion models in jumpstart support finetuning. Thus, we manually select a model
# which supports finetuning.
train_model_id, train_model_version, train_scope = (
    "model-txt2img-stabilityai-stable-diffusion-v2-1-base",
    "*",
    "training",
)

# Tested with ml.g4dn.2xlarge (16GB GPU memory) and ml.g5.2xlarge (24GB GPU memory) instances. Other instances may work as well.
# If ml.g5.2xlarge instance type is available, please change the following instance type to speed up training.
# training_instance_type = "ml.g5.2xlarge"
training_instance_type = "ml.g4dn.xlarge"   # seoul region은 g5 없으므로 g4dn 등 사용 가능

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    model_id=train_model_id,
    model_version=train_model_version,
    image_scope=train_scope,
    instance_type=training_instance_type,
)

# Retrieve the training script. This contains all the necessary files including data processing, model training etc.
train_source_uri = script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope=train_scope
)

# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = model_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, model_scope=train_scope
)

In [ ]:
print(train_image_uri)
print(train_source_uri)
print(train_model_uri)

In [ ]:
# Retrieve the default hyper-parameters for fine-tuning the model
hyper_params = hyperparameters.retrieve_default(
    model_id=train_model_id, model_version=train_model_version
)

# [Optional] Override default hyperparameters with custom values
hyper_params["max_steps"] = "200"   # 높일수록 더 해당 사진과 비슷한 형태가 되지만, overfiting 될 수 있음.
# hyper_params["with_prior_preservation"] = "True"  # it needs more memory

In [ ]:
print(hyper_params)

In [ ]:
from sagemaker.utils import name_from_base

s3_output_location = f"s3://{bucket}/lowcode-sm/logs/sd-fd-output/"
training_job_name = name_from_base(f"sd-fine-tuning-{train_model_id}")
print(training_job_name)

In [ ]:

# Create SageMaker Estimator instance
sd_estimator = Estimator(
    role=role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",  # Entry-point file in source_dir and present in train_source_uri.
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyper_params,
    output_path=s3_output_location,
    base_job_name=training_job_name,
)


In [ ]:

import os
import json
import random
import shutil

test_dir = "data/wwi-dataset/test"
label_list = os.listdir(test_dir)
print(label_list)

In [ ]:
label = "stonehenge"

dir_path = f"./sd-ft/{label}"
os.makedirs(dir_path, exist_ok=True)
instance_prompt = "A photo of my grandfather's home"
with open(os.path.join(dir_path, "dataset_info.json"), "w") as f:
    f.write(json.dumps({"instance_prompt": instance_prompt}))

test_images = os.listdir(os.path.join(test_dir, label))
random.shuffle(test_images)

for test_image in test_images[:8]:
    img_path = os.path.join(test_dir, label, test_image)
    dst_path = os.path.join(dir_path, test_image)
    shutil.copyfile(img_path, dst_path)

In [ ]:
train_s3_path = f"s3://{bucket}/lowcode-sm/sd-ft/{label}/"
!aws s3 cp --exclude ".ipynb_checkpoints*" --recursive $dir_path $train_s3_path 

In [ ]:
print(train_s3_path)

In [ ]:
# Launch a SageMaker Training job by passing s3 path of the training data
sd_estimator.fit({"training": train_s3_path}, logs=True)

In [ ]:
# training_job_name = "[training job 이름]"
# sd_estimator = Estimator.attach(training_job_name)

In [ ]:
inference_instance_type = "ml.g4dn.2xlarge"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=train_model_id,
    model_version=train_model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri. This includes scripts for model loading, inference handling etc.
deploy_source_uri = script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope="inference"
)

endpoint_name = name_from_base(f"sd-ft-{train_model_id}-")


In [ ]:
print(deploy_image_uri)
print(deploy_source_uri)


In [ ]:

# Use the estimator from the previous step to deploy to a SageMaker endpoint
finetuned_predictor = sd_estimator.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",  # entry point file in source_dir and present in deploy_source_uri
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name,
)
# print(finetuned_predictor)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def query(model_predictor, text):
    """Query the model predictor."""

    encoded_text = text.encode("utf-8")

    query_response = model_predictor.predict(
        encoded_text,
        {
            "ContentType": "application/x-text",
            "Accept": "application/json",
        },
    )
    return query_response


def parse_response(query_response):
    """Parse response and return generated image and the prompt"""

    response_dict = json.loads(query_response)
    return response_dict["generated_image"], response_dict["prompt"]


def display_img_and_prompt(img, prmpt):
    """Display hallucinated image."""
    plt.figure(figsize=(12, 12))
    plt.imshow(np.array(img))
    plt.axis("off")
    plt.title(prmpt)
    plt.show()

In [ ]:
text = "A photo of my grandfather's house"
query_response = query(finetuned_predictor, text)
img, prmpt = parse_response(query_response)
display_img_and_prompt(img, prmpt)

In [ ]:
text = "A photo of coke next to my grandfather's home"
query_response = query(finetuned_predictor, text)
img, prmpt = parse_response(query_response)
display_img_and_prompt(img, prmpt)